In [ ]:
import sys
import os
from datetime import datetime
import numpy as np
import pandas as pd
import json
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle as cp
from tensorflow import keras
import tensorflow as tf
from keras import backend as K
import datetime as dt
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
def addSpanLag(data, events=[], look_back=7, look_ahead = 7, season=0):##span==1
#     ### Adding span (required for weekly, monthly average data)
#     if span>1:
#         nts = []
#         ind = 0
#         while ind < len(ts):
#             tmpts = ts[ind:ind+span]
#             print (tmpts)
#             dt = tmpts[0]
#             vals = [k for k in tmpts]
#             pr = sum(vals)/len(vals)
#             nts.append((dt,pr))
#             ind+=span
#         ts = nts
#     if not convert_to_Xy:
#         return ts

    ### Adding lag (based on look back and look ahead)

    if len(events) > 0:
        input = np.append(data,events,axis=1)
    else:
        input = data
    if season>0:
        season = np.array([k%season for k in range(data.shape[0])])
        season = season.reshape(len(season),1)
        input = np.append(input,season,axis=1)


    ind = 0
    X_data = []
    y_data = []

    while ind + look_back + look_ahead < len(data):

        X = input[ind:ind+look_back]
        Y = data[ind+look_back+look_ahead]
        X_data.append(X)
        y_data.append(Y)
        ind+=1
    X_data = np.array(X_data)
    y_data = np.array(y_data)

    return X_data, y_data

In [ ]:
def ren_train(X, y, hidden_units=100, learning_rate=0.01, epochs=60, batch_size=256,activation='relu',\
              dropout=0.0,recurrent_dropout=0.0,regularizer='l2',optimizer='adam',kernel_regularizer='l2'):
    model = Sequential()
    model.add(LSTM(hidden_units, activation=activation,  kernel_regularizer='l2',#recurrent_regularizer='l2',
                   dropout=dropout, recurrent_dropout=recurrent_dropout,unroll=True,
                   input_shape=(X.shape[1], X.shape[-1])))
    model.add(Dense(1))
    model.compile(optimizer=optimizer, loss='mse')#,metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    history = model.fit(X, y, epochs=epochs, verbose=1)
    plt.plot(history.history['loss'])
    plt.show()
    return model

In [ ]:
# wr = open(os.path.join(path,'Oct18','output.csv'),'w')
# wr.write("crop,season,activation,drpt_recdrpt,optimizer,hidden_units,model,training_rmse,test_rmse,model,training_rmse,test_rmse,model,training_rmse,test_rmse,model,training_rmse,test_rmse,model,training_rmse,test_rmse")
# wr.close()

In [ ]:

crops = ['Onion','Potato','Wheat','Rice']
crop = crops[0]


train_split = 2557 - 1 #if using price diff
seasons = [0,12]#,365]
normalization = True

path='/content/gdrive/MyDrive/Colab Notebooks/REN/'
fileformat = path+'food_price_data/%s.csv'

output_folder ='returnrate_onedayahead'

### All together
folders = ['all_models','models_norm_mse']
dropouts = [0.0, 0.1, 0.2, 0.3]
activations = ['tanh','relu']
epochs=[60]
optimizers = ['adam']#,'rmsprop']
units = [5,10, 20, 50, 100]
kernel_regularizers = ['l2']#,None]

In [ ]:
i=1
normalization = False
season = 0

#for season in seasons:
for activation in activations:
  #for kernel_regularizer in kernel_regularizers:
    #for optimizer in optimizers:
      #for drpt in dropouts:
        #for rec_drpt in dropouts:
          for hidden_units in units:
            for crop in crops:
              epochs=70
              model_suffix = "crop=%s_season=%d_activation=%s_dropout=%.2f_optimizer=%s_units=%s_epochs=10_train=2010-15_kreg=%s"\
              %(crop,season,activation,drpt,optimizer,hidden_units,kernel_regularizer)
              print(str(i),model_suffix)
              i+=1

              food_price = pd.read_csv(fileformat%(crop))['price']
              ### price diff
              #food_price_diff = food_price.diff().dropna()
              ### Return rate
              food_price = np.log(food_price).diff()[1:]
              price_diff = np.array(food_price).reshape(food_price.shape[0],1)

              mean = np.mean(price_diff)
              std = np.std(price_diff)
              if normalization:
                  price_diff = (price_diff - mean)/std

              ### Events
              events_train = np.load(os.path.join(path,'embeddings/events_train.npy'))
              events_test = np.load(os.path.join(path,'embeddings/events_test.npy'))
              events = np.append(events_train,events_test, axis=0)[1:]

              ### Topics
              topics_train = np.load(os.path.join(path,'embeddings/toi.maxLDA_train.npy'))
              topic_test = np.load(os.path.join(path,'embeddings/toi.maxLDA_test.npy'))
              topics = np.append(topics_train,topic_test, axis=0)[1:]

              ### Word2Vec
              w2v_train = np.load(os.path.join(path,'embeddings/toi.W2V_train.npy'))
              w2v_test = np.load(os.path.join(path,'embeddings/toi.W2V_test.npy'))
              w2v = np.append(w2v_train,w2v_test, axis=0)[1:]

              ### Doc2Vec
              d2v_train = np.load(os.path.join(path,'embeddings/toi.D2V_train.npy'))
              d2v_test = np.load(os.path.join(path,'embeddings/toi.D2V_test.npy'))
              d2v = np.append(d2v_train,d2v_test, axis=0)[1:]

              X_event, y_event   = addSpanLag(price_diff,events,look_back=7,look_ahead=0,season=season)
              X_event_train = X_event[train_split:]
              y_event_train = y_event[train_split:]
              X_event_test = X_event[train_split:]
              y_event_test = y_event[train_split:]

              X_topics, y_topics   = addSpanLag(price_diff,topics,look_back=7,look_ahead=0,season=season)
              X_topics_train = X_topics[:train_split]
              y_topics_train = y_topics[:train_split]
              X_topics_test = X_topics[train_split:]
              y_topics_test = y_topics[train_split:]

              X_w2v, y_w2v   = addSpanLag(price_diff,w2v,look_back=7,look_ahead=0,season=season)
              X_w2v_train = X_w2v[:train_split]
              y_w2v_train = y_w2v[:train_split]
              X_w2v_test = X_w2v[train_split:]
              y_w2v_test = y_w2v[train_split:]

              X_d2v, y_d2v   = addSpanLag(price_diff,d2v,look_back=7,look_ahead=0,season=season)
              X_d2v_train = X_d2v[:train_split]
              y_d2v_train = y_d2v[:train_split]
              X_d2v_test = X_d2v[train_split:]
              y_d2v_test = y_d2v[train_split:]


              X_noevent, y_noevent   = addSpanLag(price_diff,look_back=7,look_ahead=0,season=season)
              X_noevent_train = X_noevent[:train_split]
              y_noevent_train = y_noevent[:train_split]
              X_noevent_test = X_noevent[train_split:]
              y_noevent_test = y_noevent[train_split:]


              ### REN
              model_event = ren_train(X_event_train,y_event_train,hidden_units=hidden_units, \
                                      activation=activation, dropout=drpt, optimizer=optimizer,epochs=epochs, kernel_regularizer=kernel_regularizer)
              model_event.save(os.path.join(path,output_folder,'ren_'+model_suffix))
              print(model_event.summary())
              ### LSTM
              model_noevent = ren_train(X_noevent_train,y_noevent_train,hidden_units=hidden_units, \
                                        activation=activation, dropout=drpt, optimizer=optimizer,epochs=epochs,kernel_regularizer=kernel_regularizer)
              model_noevent.save(os.path.join(path,output_folder,'lstm_'+model_suffix))




              others=False

              if others:
                  ### LDA
                  model_topics = ren_train(X_topics_train,y_topics_train,hidden_units=hidden_units, activation=activation, dropout=drpt, recurrent_dropout=rec_drpt,optimizer=optimizer)
                  model_topics.save(os.path.join(path,output_folder,'lda_'+model_suffix))
                  ### W2V
                  model_w2v = ren_train(X_w2v_train,y_w2v_train,hidden_units=hidden_units, activation=activation, dropout=drpt, recurrent_dropout=rec_drpt,optimizer=optimizer)
                  model_w2v.save(os.path.join(path,output_folder,'w2v_'+model_suffix))

                  ### D2V
                  model_d2v = ren_train(X_d2v_train,y_d2v_train,hidden_units=hidden_units, activation=activation, dropout=drpt, recurrent_dropout=rec_drpt,optimizer=optimizer)
                  model_d2v.save(os.path.join(path,output_folder,'d2v_'+model_suffix))

              e_res_tr = model_event.evaluate(X_event_train,y_event_train)
              e_rmse_tr = np.sqrt(e_res_tr)
              e_res = model_event.evaluate(X_event_test,y_event_test)
              e_rmse = np.sqrt(e_res)
              e_res_val = model_event.evaluate(X_event_test[:730],y_event_test[:730])
              e_rmse_val = np.sqrt(e_res_val)

              l_res_tr = model_noevent.evaluate(X_noevent_train,y_noevent_train)
              l_rmse_tr = np.sqrt(l_res_tr)
              l_res = model_noevent.evaluate(X_noevent_test,y_noevent_test)
              l_rmse = np.sqrt(l_res)
              l_res_val = model_noevent.evaluate(X_noevent_test[:730],y_noevent_test[:730])
              l_rmse_val = np.sqrt(l_res_val)


              if others:
                t_res_tr = model_topics.evaluate(X_topics_train,y_topics_train)
                t_rmse_tr = np.sqrt(t_res_tr)
                t_res = model_topics.evaluate(X_topics_test,y_topics_test)
                t_rmse = np.sqrt(t_res)

                w_res_tr = model_w2v.evaluate(X_w2v_train,y_w2v_train)
                w_rmse_tr = np.sqrt(w_res_tr)
                w_res = model_w2v.evaluate(X_w2v_test,y_w2v_test)
                w_rmse = np.sqrt(w_res)

                d_res_tr = model_d2v.evaluate(X_d2v_train,y_d2v_train)
                d_rmse_tr = (np.sqrt(d_res_tr) * std)
                d_res = model_d2v.evaluate(X_d2v_test,y_d2v_test)
                d_rmse = (np.sqrt(d_res) *std)

              wr = open(os.path.join(path,output_folder,'output.csv'),'a')
              wr.write("%s,%d,%d,%s,%s,%s,%.2f,%d,\
              events,%.3f,%.3f,%.3f,\
              lstm,%.3f,%.3f,%.3f,%s,2010-2015\n"\
              %(crop,season,epochs,activation,optimizer,kernel_regularizer,drpt,hidden_units,
                 e_rmse_tr,e_rmse_val,e_rmse,\
                 l_rmse_tr,l_rmse_val,l_rmse,\
                kernel_regularizer))
              wr.close()
              print('file saved: %s,%d,%s,%.2f,%s,%d'%(crop,season+10,activation,drpt,optimizer,hidden_units))

            # lda,%.3f,%.3f,\
              # w2v,%.3f,%.3f,\
              # d2v,%.3f,%.3f,\
              # t_rmse_tr,t_rmse,\
                #  w_rmse_tr,w_rmse,d_rmse_tr,d_rmse,\